# Word2Vec

Word2Vec skip-gram model (see [Mikolov,Sutskever,Chen,Dean paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)) over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Found and verified text8.zip
Data size 17005207
Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5237, 3082, 12, 6, 195, 2, 3136, 46, 59, 156]


In [2]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['as', 'anarchism', 'a', 'originated', 'as', 'term', 'of', 'a']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['term', 'originated', 'a', 'anarchism', 'originated', 'of', 'term', 'as']


In [3]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar(var.name+'_mean', mean)
    #tf.scalar_summary(var.name+'_mean', mean)
    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar(var.name+'_stddev', stddev)
    #tf.scalar_summary(var.name+'_stddev', stddev)
    tf.summary.scalar(var.name+'_max', tf.reduce_max(var))
    #tf.scalar_summary(var.name+'_max', tf.reduce_max(var))
    tf.summary.scalar(var.name+'_min', tf.reduce_min(var))
    #tf.histogram_summary( var.name, var)
    tf.summary.histogram( var.name, var)

In [4]:
def save_metadata(file,d):
        with open(file, 'w') as f:
            for i in range(len(d)):
                c = d[i]
                f.write('{}\n'.format(c))

In [5]:
from tensorflow.contrib.tensorboard.plugins import projector
config = projector.ProjectorConfig()

batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  #with tf.name_scope('embed'):
  embeddings = tf.Variable(
      tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0) , name="embeddings")
  #variable_summaries(embeddings)

  embedding = config.embeddings.add()
  embedding.tensor_name = embeddings.name
  # Link this tensor to its metadata file (e.g. labels).
  embedding.metadata_path = "./_logs3/metadata.tsv"

  if not os.path.exists("./_logs3/"):
    os.makedirs("./_logs3/")
  save_metadata(file="./_logs3/metadata.tsv",d=list(reverse_dictionary.values()))
    
  
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  
  with tf.name_scope('loss_function'):
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed,
                               train_labels, num_sampled, vocabulary_size))
    tf.summary.scalar('loss', loss)

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  

  with tf.name_scope('similarity_function'):
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))
  variable_summaries(similarity)

INFO:tensorflow:Summary name similarity_function/MatMul:0_mean is illegal; using similarity_function/MatMul_0_mean instead.
INFO:tensorflow:Summary name similarity_function/MatMul:0_stddev is illegal; using similarity_function/MatMul_0_stddev instead.
INFO:tensorflow:Summary name similarity_function/MatMul:0_max is illegal; using similarity_function/MatMul_0_max instead.
INFO:tensorflow:Summary name similarity_function/MatMul:0_min is illegal; using similarity_function/MatMul_0_min instead.
INFO:tensorflow:Summary name similarity_function/MatMul:0 is illegal; using similarity_function/MatMul_0 instead.


In [6]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    
  saver = tf.train.Saver()  
    
    
  tf.global_variables_initializer().run()
  print('Initialized')
    
  merged = tf.summary.merge_all()
  writer = tf.summary.FileWriter('./_logs3',session.graph)  
    
  # Saves a configuration file that TensorBoard will read during startup.
  projector.visualize_embeddings(writer, config)
    
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
      
      summary,_ = session.run([merged,similarity], feed_dict=feed_dict)
      writer.add_summary(summary, step)
      writer.flush()
          
      checkpoint_file = os.path.join('./_logs3', 'checkpoint')
      saver.save(session, checkpoint_file, global_step=i)  
    
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 7.954560
Nearest to also: pausanias, mercantilism, shortened, christological, bw, herding, nils, commons,
Nearest to so: greenwich, marten, landau, menace, bloody, explains, konar, potency,
Nearest to more: ebro, foy, translator, valuation, wakko, jacobsen, ammi, dopant,
Nearest to about: pellets, minor, speed, eero, regular, untranslated, overturn, monitoring,
Nearest to UNK: dispensation, vantage, receiver, cropped, entitled, converging, stagnation, abkhazian,
Nearest to five: scrupulous, stefan, warhol, ago, selangor, mutant, braid, specifically,
Nearest to state: enos, dong, adoptionism, ivanhoe, bridgehead, furthering, riga, cohorts,
Nearest to if: maximize, yahweh, barkley, cruisers, linseed, garland, abomination, fiddle,
Nearest to however: collimated, tethys, septuagint, doodle, pot, gi, explanation, lyell,
Nearest to on: dared, extraneous, crystallization, arguing, ignaz, extend, watches, ditches,
Nearest to these: fdr, knesset, kurosawa, rc

## Embedding Visualization
Embeddings are ubiquitous in machine learning, appearing in recommender systems, NLP, and many other applications. Indeed, in the context of TensorFlow, it's natural to view tensors (or slices of tensors) as points in space, so almost any TensorFlow system will naturally give rise to various embeddings.

To learn more about embeddings and how to train them, see the [Vector Representations of Words](https://www.tensorflow.org/tutorials/word2vec/) tutorial. If you are interested in embeddings of images, check out [this article](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) for interesting visualizations of MNIST images. On the other hand, if you are interested in word embeddings, [this article](http://colah.github.io/posts/2015-01-Visualizing-Representations/) gives a good introduction

### PCA
To run TensorBoard, use the following command (alternatively python -m tensorflow.tensorboard)
>tensorboard --logdir=_logs3

<img src="images/embed1.JPG" />

We can see that closest neighbors of __neighbors__ are 

* __lisp__ 0.903
* __interestingly__ 0.910
* __exceeds__ 0.935
* __bulky__ 0.946



### t-SNA

<img src="images/embed2.JPG" />